In [1]:
pip install sanpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
# API key - qovlkte53ks5zjha_jcn4x4363pnaroe2
from san import *
import san
from decimal import Decimal
import pandas as pd
import datetime

In [2]:
import san
san.ApiConfig.api_key = 'qovlkte53ks5zjha_jcn4x4363pnaroe2'
san.get("projects/all")

,marketSegment,name,slug,ticker,totalSupply
0,Platform,Color Platform,color-platform,CLR,350000000
1,Ethereum,Enecuum,enecuum,ENQ,193716760.116
2,Interoperability,Cosmos,cosmos,ATOM,268426508.255299
3,Ethereum,ZeuxCoin,zeuxcoin,ZUC,750000000
4,Gaming,Seed,seed,SEED,None
...,...,...,...,...,...
1939,Polkadot,Crust,crust,CRU,2000010
1940,Governance,PieDAO DOUGH v2,piedao-dough-v2,DOUGH,76629009.02192521
1941,DeFi,SmartKey,smartkey,SKEY,0
1942,Privacy,Firo,zcoin,XZC,21400000


In [3]:
# Getting all slugs/tickers/names/marketsegment/supply and dropping all with missing or 0 supply
data = san.get("projects/all")
data = data.dropna()
data['totalSupply'] = data['totalSupply'].astype(float)
data = data[data['totalSupply'] != 0]

In [4]:
# Limiting only to Cryptocurrency
crypto = data[data["marketSegment"] == "Cryptocurrency"]
crypto = crypto.sort_values(by = ["totalSupply"], ascending = False)
crypto

,marketSegment,name,slug,ticker,totalSupply
1749,Cryptocurrency,Dimecoin,dimecoin,DIME,5.559981e+11
1181,Cryptocurrency,Mooncoin,mooncoin,MOON,2.274909e+11
1855,Cryptocurrency,NewYorkCoin,newyorkcoin,NYC,1.437611e+11
1708,Cryptocurrency,Dogecoin,dogecoin,DOGE,1.291688e+11
1770,Cryptocurrency,eXPerience Chain,experience-chain,XPC,1.132155e+11
...,...,...,...,...,...
1819,Cryptocurrency,Bitcoin Rhodium,bitcoin-rhodium,XRC,1.316162e+06
1539,Cryptocurrency,Sora,sora,XOR,3.500000e+05
1443,Cryptocurrency,ParallelCoin,parallelcoin,DUO,3.147186e+05
347,Cryptocurrency,Unobtanium,unobtanium,UNO,2.033284e+05


In [5]:
def metriclist(slug):
    metric = "marketcap_usd" 
    fullname = metric + "/" + slug 
    metrics = san.get(
    fullname,
    from_date="2018-04-01",
    to_date="2021-04-01",
    interval="1h"
    )
    newname = slug+"_"+metric
    finaldf = metrics.rename(columns={"value": newname})
    return(finaldf)

In [6]:
# Change metric and slug fullname = metric + slug
# Running through all cryptocurrencies and adding price and daily returns
count = 0
metric_df = pd.DataFrame()
for i in crypto['slug'].tolist():
    holddf = metriclist(i)
    if count == 0:
        metric_df = holddf
    else:
        metric_df = pd.merge(metric_df, holddf, how = "left", on = "datetime")
    count = count+1

In [7]:
metric_df

,dimecoin_marketcap_usd,mooncoin_marketcap_usd,newyorkcoin_marketcap_usd,dogecoin_marketcap_usd,experience-chain_marketcap_usd,telcoin_marketcap_usd,ripple_marketcap_usd,vipstar-coin_marketcap_usd,microbitcoin_marketcap_usd,inflationcoin_marketcap_usd,...,lightpaycoin_marketcap_usd,axe_marketcap_usd,buysell_marketcap_usd,stakinglab_marketcap_usd,novacoin_marketcap_usd,bitcoin-rhodium_marketcap_usd,sora_marketcap_usd,parallelcoin_marketcap_usd,unobtanium_marketcap_usd,amoveo_marketcap_usd
datetime,,,,,,,,,,,,,,,,,,,,,
2018-04-01 00:00:00+00:00,14487685.0,19195596.0,9584301.0,3.245819e+08,NaN,49614406.0,2.009591e+10,NaN,NaN,2239172.0,...,NaN,NaN,NaN,NaN,6632308.0,NaN,NaN,113366.0,27979525.0,NaN
2018-04-01 01:00:00+00:00,14647797.0,15681738.0,15552753.0,3.205742e+08,NaN,51495866.0,2.008633e+10,NaN,NaN,2244797.0,...,NaN,NaN,NaN,NaN,6626933.0,NaN,NaN,113610.0,27983117.0,NaN
2018-04-01 02:00:00+00:00,14427199.0,15611555.0,9470802.0,3.176080e+08,NaN,50721613.0,1.991491e+10,NaN,NaN,2215207.0,...,NaN,NaN,NaN,NaN,6568450.0,NaN,NaN,112284.0,27726981.0,NaN
2018-04-01 03:00:00+00:00,14444989.0,22568931.0,9509071.0,3.206914e+08,NaN,47957273.0,1.986381e+10,NaN,NaN,2217015.0,...,NaN,NaN,NaN,NaN,6586657.0,NaN,NaN,112440.0,27306342.0,NaN
2018-04-01 04:00:00+00:00,14353558.0,15592003.0,9431104.0,3.206083e+08,NaN,50703791.0,1.990916e+10,NaN,NaN,1113014.0,...,NaN,NaN,NaN,NaN,6555489.0,NaN,NaN,112218.0,27250022.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-01 19:00:00+00:00,6252759.0,NaN,0.0,7.552568e+09,10626712.0,402922640.0,2.605054e+10,2142554.0,1902869.0,0.0,...,NaN,1470963.0,0.0,2160.0,NaN,8096750.0,201414683.0,187777.0,0.0,0.0
2021-04-01 20:00:00+00:00,5048781.0,NaN,0.0,7.739841e+09,12632277.0,407575678.0,2.605754e+10,2256690.0,1899175.0,0.0,...,NaN,1445566.0,0.0,2159.0,NaN,7991975.0,200921856.0,187899.0,0.0,0.0
2021-04-01 21:00:00+00:00,6368081.0,NaN,0.0,7.649151e+09,13113001.0,405064414.0,2.600472e+10,2311815.0,1893407.0,0.0,...,NaN,1476040.0,0.0,2155.0,NaN,8257852.0,200996809.0,187656.0,0.0,0.0
